# Introduction

This analysis presents a complete customer segmentation and predictive analytics workflow using transaction data

The goal is to convert raw purchase data into actionable insights that inform marketing strategies, improve customer retention, and support revenue forecasting.


## 1. Dataset Source and Description

The dataset used in this project was obtained from the **UC Irvine Machine Learning Repository**:

**Online Retail II Dataset**  
Donated: September 20, 2019  
Source: UC Irvine Machine Learning Repository  

🔗 [https://archive.ics.uci.edu/dataset/502/online+retail+ii](https://archive.ics.uci.edu/dataset/502/online+retail+ii)

This dataset contains **two years of real online retail transactions** from a UK-based, non-store online retailer, covering the period **December 2009 to December 2011**. The company primarily sells unique, all-occasion gift products, with many customers being wholesalers.

The dataset is well-suited for **customer analytics and predictive modeling**, and supports tasks such as:

- Classification  
- Regression  
- Clustering  

It includes a mix of **transactional, temporal, and categorical features**, making it ideal for RFM analysis, customer segmentation, and CLV modeling.

#### Variable Description

| Fields/ Columns       | Description |
|-------------|------------|
| InvoiceNo    | Unique invoice number for each transaction. If it starts with "C", it indicates a cancellation. |
| StockCode    | Unique product identifier. |
| Description  | Product name. |
| Quantity     | Number of items purchased in a transaction. |
| InvoiceDate  | Date and time when the transaction occurred. |
| UnitPrice    | Price per unit in British Pounds (£). |
| CustomerID   | Unique customer identifier. |
| Country      | Customer’s country of residence. |


## 2. Imports

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import os
import openpyxl

## 3. Load Config

In [16]:
with open("../config.yaml","r") as file:
    config = yaml.safe_load(file)

data_path = os.path.join("..", config["paths"]["raw_data"])

## 4. Data Loading & Exploration (EDA)

### Dataset Overview:

The dataset contains two yearly transaction sheets, representing consecutive retail periods.

 - Year 2009-2010
 - Year 2010-2011

We shall be merging it later if the fields match 

**Below are the content breakdown for the different Fields**

| Column      | Notes                                                           |
| ----------- | --------------------------------------------------------------- |
| Invoice     | Object – includes normal, cancellation, and adjustment invoices |
| StockCode   | Mixed formats                                                   |
| Description | Some missing values                                             |
| Quantity    | Contains negative values                                        |
| InvoiceDate | Proper datetime                                                 |
| Price       | Contains negative values                                        |
| Customer ID | ~243k missing                                                   |
| Country     | Mostly UK                                                       |

> **Loading the Data**

In [18]:
dfs = pd.read_excel(data_path, sheet_name=None)  # None = load all sheets
print("Sheets loaded:", list(dfs.keys()))

df1 = dfs[list(dfs.keys())[0]]  # first tab
df2 = dfs[list(dfs.keys())[1]]  # second tab

print(df1.head(3))
print(df2.head(3))

Sheets loaded: ['Year 2009-2010', 'Year 2010-2011']
  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   

          InvoiceDate  Price  Customer ID         Country  
0 2009-12-01 07:45:00   6.95      13085.0  United Kingdom  
1 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
2 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
  Invoice StockCode                         Description  Quantity  \
0  536365    85123A  WHITE HANGING HEART T-LIGHT HOLDER         6   
1  536365     71053                 WHITE METAL LANTERN         6   
2  536365    84406B      CREAM CUPID HEARTS COAT HANGER         8   

          InvoiceDate  Price  Customer ID         Country  
0 2010-12-01 08:26:00   2.55      17850.0  United Kingdom  
1 2010-12-01 08:26:00   3.39  

> **Merging Datasets**
 - Both years share identical structure, so we safely merge them into a single transactional dataset.

In [ ]:
# Checking if the columns are the same then merging
columns_match = df1.columns.equals(df2.columns)
print("Columns Match:", columns_match)

if columns_match:
    df = pd.concat([df1, df2], axis=0, ignore_index=True)
    print(df.head(3))
else:
    print("Columns do not match. Please check the data.")

Columns Match: True
  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   

          InvoiceDate  Price  Customer ID         Country  
0 2009-12-01 07:45:00   6.95      13085.0  United Kingdom  
1 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
2 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  


#### Data Exploration:

**a) Data Overview**

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  object        
 1   StockCode    1067371 non-null  object        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   Price        1067371 non-null  float64       
 6   Customer ID  824364 non-null   float64       
 7   Country      1067371 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 65.1+ MB


**b) Numerical Summary**

- **Business Meaning**
    - Issue & Interpretation 
        - Negative Quantity : Returns / cancellations 
        - Negative Price    : Adjustments / accounting corrections
        - Extreme values    : Non-sales financial entries

In [ ]:
df.describe() # descriptive statistics for numerical columns

,Quantity,InvoiceDate,Price,Customer ID
count,1.067371e+06,1067371,1.067371e+06,824364.000000
mean,9.938898e+00,2011-01-02 21:13:55.394028544,4.649388e+00,15324.638504
min,-8.099500e+04,2009-12-01 07:45:00,-5.359436e+04,12346.000000
25%,1.000000e+00,2010-07-09 09:46:00,1.250000e+00,13975.000000
50%,3.000000e+00,2010-12-07 15:28:00,2.100000e+00,15255.000000
75%,1.000000e+01,2011-07-22 10:23:00,4.150000e+00,16797.000000
max,8.099500e+04,2011-12-09 12:50:00,3.897000e+04,18287.000000
std,1.727058e+02,NaN,1.235531e+02,1697.464450


**c). Categorical Summary**
 - 53,628 invoices
 - 5,305 products
- 43 countries
- UK dominates dataset (~92%)

In [26]:
df.describe(include='O') # descriptive statistics for categorical columns

,Invoice,StockCode,Description,Country
count,1067371,1067371,1062989,1067371
unique,53628,5305,5698,43
top,537434,85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom
freq,1350,5829,5918,981330


**d). Missing Customer IDs**
 - This matters because Without Customer IDs we cannot assign RFM or CLV thus these records must be excluded from segmentation.

In [28]:
print("Null values in Customer ID column:", df["Customer ID"].isna().sum())
display(df[df["Customer ID"].isna()].head(5))

Null values in Customer ID column: 243007


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
263,489464,21733,85123a mixed,-96,2009-12-01 10:52:00,0.00,NaN,United Kingdom
283,489463,71477,short,-240,2009-12-01 10:52:00,0.00,NaN,United Kingdom
284,489467,85123A,21733 mixed,-192,2009-12-01 10:53:00,0.00,NaN,United Kingdom
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.00,NaN,United Kingdom
577,489525,85226C,BLUE PULL BACK RACING CAR,1,2009-12-01 11:49:00,0.55,NaN,United Kingdom


**e). Negative Quantities:** 
 - This means these records are Returns, cancellations, or corrections.

In [36]:
print("Negative values in Quantity column:", df["Quantity"].lt(0).sum())
display(df[df['Quantity'] < 0].head(3))

Negative values in Quantity column: 22950


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.0,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.0,Australia


#### Invoice Structure Analysis: 
 - Goal: 
    - Checking to see if invoice has anything other than 6 digit number.
    - starts with C which is cancellation

- Observation: 
    - We have 19,500 invoices with more than 6 digits
    - C , and A are the starting letters on the invoices with more than 6 digits 

- Implication: 
     - The rows with lettersrepresent financial corrections, not purchases. i.e 
        - None : Normal sales
        - C    : Sales Cancellations (19,494)
        - A    : Sales Adjustments (6)

In [40]:
df["Invoice"] = df["Invoice"].astype("str") # converting Invoice column to string

# looking at invoices with more than 6 digits
print("Invoices with more than 6 digits:", df["Invoice"].str.len().gt(6).sum())
print("Snapshot of Invoices with more than 6 digits:")
display(df[df["Invoice"].str.len() > 6].head(3))

Invoices with more than 6 digits: 19500
Snapshot of Invoices with more than 6 digits:


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.0,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.0,Australia


In [ ]:
# 2. checking if all invoices start with letter C
df["Invoice"].str.replace("[0-9]", "", regex=True).unique() # removing digits

array(['', 'C', 'A'], dtype=object)

In [43]:
# Count of Records that are cancellations and adjustments
print("Count of Records that are cancellations and adjustments:", df[df["Invoice"].str.startswith(("C", "A"))].shape[0])
print("Count of Cancellation Records:", df[df["Invoice"].str.startswith("C")].shape[0])
print("Count of Adjustment Records:", df[df["Invoice"].str.startswith("A")].shape[0])

Count of Records that are cancellations and adjustments: 19500
Count of Cancellation Records: 19494
Count of Adjustment Records: 6


#### StockCode Pattern Analysis

**StockCode Interpretation Table**

| Code             | Description            | Action  |
| ---------------- | ---------------------- | ------- |
| DCGS*            | Gift sets / bundles    | Exclude |
| D                | Discount               | Exclude |
| DOT              | Postage                | Exclude |
| M / m            | Manual entry           | Exclude |
| C2 / C3          | Carriage               | Exclude |
| BANK CHARGES / B | Bank fees              | Exclude |
| S                | Samples                | Exclude |
| TEST*            | Testing                | Exclude |
| gift_*           | Gift cards             | Exclude |
| PADS             | Padding product        | Include |
| SP1002           | Special product        | Exclude |
| AMAZONFEE        | Amazon fees            | Exclude |
| ADJUST*          | Accounting adjustments | Exclude |
| CRUK             | Charity donation       | Exclude |


**a). All StockCodes**

In [ ]:
# Looking at stock codes besides 5 digits and 5 digits with letter at the end
df["StockCode"] = df["StockCode"].astype("str")

stock_codes = df[(df["StockCode"].str.match("^\\d{5}$") == False) & 
                   (df["StockCode"].str.match("^\\d{5}[a-zA-Z]+$") == False)
                   ]["StockCode"].unique()

stock_codes

array(['POST', 'D', 'DCGS0058', 'DCGS0068', 'DOT', 'M', 'DCGS0004',
       'DCGS0076', 'C2', 'BANK CHARGES', 'DCGS0003', 'TEST001',
       'gift_0001_80', 'DCGS0072', 'gift_0001_20', 'DCGS0044', 'TEST002',
       'gift_0001_10', 'gift_0001_50', 'DCGS0066N', 'gift_0001_30',
       'PADS', 'ADJUST', 'gift_0001_40', 'gift_0001_60', 'gift_0001_70',
       'gift_0001_90', 'DCGSSGIRL', 'DCGS0006', 'DCGS0016', 'DCGS0027',
       'DCGS0036', 'DCGS0039', 'DCGS0060', 'DCGS0056', 'DCGS0059', 'GIFT',
       'DCGSLBOY', 'm', 'DCGS0053', 'DCGS0062', 'DCGS0037', 'DCGSSBOY',
       'DCGSLGIRL', 'S', 'DCGS0069', 'DCGS0070', 'DCGS0075', 'B',
       'DCGS0041', 'ADJUST2', '47503J ', 'C3', 'SP1002', 'AMAZONFEE',
       'DCGS0055', 'DCGS0074', 'DCGS0057', 'DCGS0073', 'DCGS0071',
       'DCGS0066P', 'DCGS0067', 'CRUK'], dtype=object)

In [35]:
df[df["StockCode"].str.contains("^DOT")] # looking at stock codes with DOT in them

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
2379,489597,DOT,DOTCOM POSTAGE,1,2009-12-01 14:28:00,647.19,NaN,United Kingdom
2539,489600,DOT,DOTCOM POSTAGE,1,2009-12-01 14:43:00,55.96,NaN,United Kingdom
2551,489601,DOT,DOTCOM POSTAGE,1,2009-12-01 14:44:00,68.39,NaN,United Kingdom
2571,489602,DOT,DOTCOM POSTAGE,1,2009-12-01 14:45:00,59.35,NaN,United Kingdom
2619,489603,DOT,DOTCOM POSTAGE,1,2009-12-01 14:46:00,42.39,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
1062295,581219,DOT,DOTCOM POSTAGE,1,2011-12-08 09:28:00,1008.96,NaN,United Kingdom
1062715,581238,DOT,DOTCOM POSTAGE,1,2011-12-08 10:53:00,1683.75,NaN,United Kingdom
1064829,581439,DOT,DOTCOM POSTAGE,1,2011-12-08 16:30:00,938.59,NaN,United Kingdom
1066369,581492,DOT,DOTCOM POSTAGE,1,2011-12-09 10:03:00,933.17,NaN,United Kingdom


## 5. Data Cleaning for RFM & CLV